In [1]:
%load_ext autoreload
%autoreload 2

from DDR import process_ddr
import pandas as pd
import dill as pickle
from datetime import datetime
from data_prep import get_efd_rf
from updates import prepare_dataset_with_updates
from tqdm import tqdm

In [ ]:
archive = r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS\ddrunpacked_full2.csv"
capacity_df = pd.read_csv(archive)
capacity_df

In [ ]:
import itertools
archive= r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS"

taf_archive = r"C:\Users\iLabs_6\Documents\Tex\allwithCBAS\taf_reports.pkl"

with open(taf_archive, 'rb') as f:
    taf_ar = pickle.load(f)
#

start_date = datetime(2023, 3, 24, 0, 0)
end_date = datetime(2023, 10, 28, 0, 0)


efd = get_efd_rf(archive, start_date, end_date, taf_reports=taf_ar, reg_reports=None, reload=True)


In [ ]:
archive = r"C:\Users\iLabs_6\Documents\Tex\realtimetest3\newbaseline_atot2.csv"
try:
    extended_df = pd.read_csv(archive)
    extended_df['distance'] = extended_df['distance'].astype(float)
    extended_df['atfmdelay'] = extended_df['atfmdelay'].astype(float)
    cols = extended_df.columns.tolist()
    datetime_cols = ['ETOT', 'EOBT', 'ETA', 'cbasentry', 'TSAT', 'TOBT']
    timedelta_cols =[col for col in extended_df.columns if 'CBAS' in col]
    for col in datetime_cols:
        extended_df[col] = pd.to_datetime(extended_df[col], errors='coerce')
    for col in timedelta_cols:
        extended_df[col] = pd.to_timedelta(extended_df[col], errors='coerce')
 
except Exception as e:
    print(f'{e=}')
    extended_df = prepare_dataset_with_updates(efd, cap=capacity_df, horizon_hours=5, dt_minutes=5, taf_ar=taf_ar, save_ar= archive, mode='atot')


In [ ]:
import os
output= r"C:\Users\iLabs_6\Documents\Tex\realtimetest3"

modelname = 'newbaseline_etot2'
with open(os.path.join(output, modelname), 'wb') as f:  
    pickle.dump(extended_df, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"FlightDataset has been saved successfully. {modelname}")

